In [ ]:
import pandas as pd
import subprocess
import tempfile
import os
import json
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

solutions_train = pd.read_excel("data/for_teams/train/solutions.xlsx")
solutions_test = pd.read_excel("data/for_teams/test/solutions.xlsx")

In [ ]:
def run_pyright_on_string(code_string: str):
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
            temp_file_path = temp_file.name
            temp_file.write(code_string.encode("utf-8"))

        result = subprocess.run(
            ["basedpyright", "--outputjson", "--level", "error", "--project", ".", temp_file_path],
            capture_output=True,
            text=True,
        )

    except FileNotFoundError:
        print("Pyright is not installed or not found in your PATH.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)
            print(f"Temporary file {temp_file_path} deleted.")

    return result.stdout


def get_basedpyright_errors(code_string: str):
    basedpyright_output = run_pyright_on_string(code_string)
    lines = code_string.split("\n")
    errors = json.loads(basedpyright_output)["generalDiagnostics"]
    count = len(errors)
    if count == 0:
        return {"message": None, "line_number": None, "line": None}
    errors = errors[0]

    return {
        "message": errors["message"],
        "line_number": errors["range"]["start"]["line"],
        "line": lines[errors["range"]["start"]["line"]],
    }

In [ ]:
solutions_train[["message", "line_number", "line"]] = solutions_train["student_solution"].parallel_apply(
    lambda x: pd.Series(get_basedpyright_errors(x))
)

solutions_train.to_csv("train_solutions_with_pyright.csv")

In [ ]:
solutions_test[["message", "line_number", "line"]] = solutions_test["student_solution"].parallel_apply(
    lambda x: pd.Series(get_basedpyright_errors(x))
)

solutions_test.to_csv("test_solutions_with_pyright.csv")